In [1]:
import sqlite3
import os
from pathlib import Path as pth
import subprocess as sp
import pandas as pd
import numpy as np
import re
import unicodedata
from typing import List, Tuple, Any

In [2]:
os.chdir(pth(pth.home() / 'dev/sms-analysis'))

In [3]:
# copy chat db so no funky stuff
sp.run(["scp","/Users/canderson/Library/Messages/chat.db", "raw-data/copy-of-chat.db"])

CompletedProcess(args=['scp', '/Users/canderson/Library/Messages/chat.db', 'raw-data/copy-of-chat.db'], returncode=0)

In [4]:
# Connect to SQLite Database and create a cursor
db_path = pth('raw-data/copy-of-chat.db')
sqliteConnection = sqlite3.connect(db_path)

In [5]:
def query(sql: str) -> List[Tuple[Any, ...]]:
    """
    Return the result of a SQLite query using the existing connection.

    Parameters
    ----------
    sql : str
        SQL query formatted as a text block.
    """
    cursor = sqliteConnection.cursor()
    cursor.execute(sql)
    res = cursor.fetchall()
    cursor.close()
    return res

In [6]:
query('SELECT sqlite_version();')

[('3.51.1',)]

In [7]:
# Tables
query("""
SELECT name
FROM sqlite_master
WHERE type = 'table'
ORDER BY name;
""")

[('_SqliteDatabaseProperties',),
 ('attachment',),
 ('chat',),
 ('chat_handle_join',),
 ('chat_message_join',),
 ('chat_recoverable_message_join',),
 ('deleted_messages',),
 ('handle',),
 ('kvtable',),
 ('message',),
 ('message_attachment_join',),
 ('message_processing_task',),
 ('recoverable_message_part',),
 ('scheduled_messages_pending_cloudkit_delete',),
 ('sqlite_sequence',),
 ('sqlite_stat1',),
 ('sync_deleted_attachments',),
 ('sync_deleted_chats',),
 ('sync_deleted_messages',),
 ('unsynced_removed_recoverable_messages',)]

In [8]:
query_response = query("""
    SELECT
        datetime (message.date / 1000000000 + strftime ("%s", "2001-01-01"), "unixepoch", "localtime") AS message_date,
        message.text,
        message.is_from_me,
        chat.chat_identifier
    FROM
        chat
        JOIN chat_message_join ON chat. "ROWID" = chat_message_join.chat_id
        JOIN message ON chat_message_join.message_id = message. "ROWID"
    ORDER BY
        message_date ASC;
""")

In [9]:
sqliteConnection.close()

In [10]:
# make df
all_messages = pd.DataFrame(query_response, columns = ['date_time', 'text', 'from_me', 'sender'])

In [11]:
all_messages.shape

(41771, 4)

In [12]:
# filter for empty messages
s = all_messages.loc[8407, "text"]
for i, ch in enumerate(s):
    print(
        i,
        repr(ch),
        f"U+{ord(ch):04X}",
        unicodedata.name(ch, "UNKNOWN"),
        unicodedata.category(ch)
    )

0 '￼' U+FFFC OBJECT REPLACEMENT CHARACTER So


In [13]:
messages = all_messages

In [14]:
messages['text'] = (
    messages['text']
    .str.replace('\uFFFC', '', regex = False)
)

In [15]:
# filter out empty cells
messages = messages[ messages.text.str.strip().ne("")]

In [16]:
# filter out None rows
messages = messages[messages.text.notna()]

In [17]:
# filter for katrina messages
X_messages = messages[messages.sender.str.contains('30761',na = False)]

In [18]:
# make date class
X_messages = X_messages.copy()
X_messages['date_time'] = pd.to_datetime(X_messages["date_time"])

In [19]:
X_messages = X_messages.copy()
X_messages=X_messages.sort_values("date_time").reset_index(drop=True)

In [20]:
print(X_messages.shape)
X_messages.tail(10)

(9766, 4)


,date_time,text,from_me,sender
9756,2025-12-25 18:06:58,lol you scared of her?,0,+13608307613
9757,2025-12-25 18:07:11,Kinds disturbing haha,0,+13608307613
9758,2025-12-25 18:08:37,I know! Just messing with you,0,+13608307613
9759,2025-12-25 18:08:37,This looks so fun!,0,+13608307613
9760,2025-12-25 18:10:20,I would loooove that,0,+13608307613
9761,2025-12-25 18:10:22,You look very cute,0,+13608307613
9762,2025-12-25 18:11:40,I think it’s hitting me,0,+13608307613
9763,2025-12-25 18:11:41,How much I miss you,0,+13608307613
9764,2025-12-25 19:00:05,I’m still thinking it will be around the same ...,0,+13608307613
9765,2025-12-25 19:00:09,Around 10pm my time,0,+13608307613


In [21]:
X_messages.to_csv('processed-data/X-messages.csv',index = False)

In [22]:
with open("processed-data/messages.txt", "w", encoding="utf-8") as f:
    f.write("\n----------\n".join(
        X_messages["date_time"].astype(str) + ":: " + X_messages["from_me"].astype(str) + ":: " + X_messages["text"].astype(str)
        ))

In [23]:
# clean empty lines
with open("processed-data/messages.txt", encoding="utf-8") as f:
    lines = [l for l in f if l.strip()]

In [24]:
with open("processed-data/messages.txt", "w", encoding="utf-8") as f:
    f.writelines(lines)